In [ ]:
import pandas as pd
from sqlalchemy import create_engine 

### Store CSV Data into DataFrame

In [ ]:
csv_file = "Resources/items.csv"
phone_data_df = pd.read_csv(csv_file)
phone_data_df.head()

In [ ]:
csv_file = "Resources/reviews.csv"
reviews_data_df = pd.read_csv(csv_file)
reviews_data_df.head()

### Create new data with select columns

In [ ]:
new_phone_data_df=phone_data_df[['asin','brand','title','price']].copy()
new_phone_data_df.head()

In [ ]:
new_reviews_data_df=reviews_data_df[['asin','rating']].copy()
new_reviews_data_df.head()

### Clean DataFrame

##### Remove NAN


In [ ]:
new_phone_data_df=new_phone_data_df.dropna()
new_phone_data_df.head()

##### Remove phones with no price

In [ ]:
new_phone_data_df=new_phone_data_df.loc[new_phone_data_df['price']!=0]
new_phone_data_df.head()

##### Rename dataframe columns

In [ ]:
new_phone_data_df=new_phone_data_df.rename(columns={'asin':'id','brand':'phone_brand','title':'phone_model','price':'phone_price'})
new_phone_data_df.head()

In [ ]:
new_reviews_data_df=new_reviews_data_df.rename(columns={'asin':'id'})
new_reviews_data_df.head()

### Aggregate reviews data

In [ ]:
group_new_reviews_data_df=new_reviews_data_df.groupby('id').mean().reset_index()
#rename
group_new_reviews_data_df=group_new_reviews_data_df.rename(columns={'rating':'avg_rating'})
#Round off Avg rating to 1 decimal place
group_new_reviews_data_df=group_new_reviews_data_df.round({'avg_rating':1})
group_new_reviews_data_df.head()



### Merge dataframes 

In [ ]:
phone_rating_df=pd.merge(new_phone_data_df,group_new_reviews_data_df, on='id')
phone_rating_df.head()

### Connect to local database

In [ ]:
 conn = "postgres:postgres@localhost:5432/Phone_db"
engine = create_engine(f'postgresql://{conn}')

##### Check for tables

In [ ]:
engine.table_names()

### Load DataFrame into database

In [ ]:
phone_rating_df.to_sql(name='phone_rating', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from phone_rating',con=engine).head()